In [1]:
import pandas as pd
import numpy as np
import warnings
import nltk


C:\Users\Sm\Anaconda3\lib\site-packages\sklearn\feature_extraction\dict_vectorizer.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping


In [2]:
import contractions
from bs4 import BeautifulSoup
import unicodedata
import re
import nltk
import numpy as np

ps = nltk.porter.PorterStemmer()
stop_words = nltk.corpus.stopwords.words('english')
stop_words.remove('no')
stop_words.remove('but')
stop_words.remove('not')


def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text


def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text


def expand_contractions(text):
    return contractions.fix(text)


def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text


def remove_stopwords(text, is_lower_case=False, stopwords=None):
    if not stopwords:
        stopwords = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopwords]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text


def pre_process_document(document):
    
    # strip HTML
    document = strip_html_tags(document)
    
    # lower case
    document = document.lower()
    
    # remove extra newlines (often might be present in really noisy text)
    document = document.translate(document.maketrans("\n\t\r", "   "))
    
    # remove accented characters
    document = remove_accented_chars(document)
    
    # expand contractions    
    document = expand_contractions(document)
               
    # remove special characters and\or digits    
    # insert spaces between special characters to isolate them    
    special_char_pattern = re.compile(r'([{.(-)!}])')
    document = special_char_pattern.sub(" \\1 ", document)
    document = remove_special_characters(document, remove_digits=True)        
    
    # remove stopwords
    document = remove_stopwords(document, is_lower_case=True, stopwords=stop_words)
        
    # remove extra whitespace
    document = re.sub(' +', ' ', document)
    document = document.strip()
    
    return document


pre_process_corpus = np.vectorize(pre_process_document)

In [3]:
dataset = pd.read_csv('movie_reviews.csv.bz2', compression='bz2')

# take a peek at the data
print(dataset.head())
reviews = np.array(dataset['review'])
sentiments = np.array(dataset['sentiment'])

# build train and test datasets
train_reviews = reviews[:35000]
train_sentiments = sentiments[:35000]
test_reviews = reviews[35000:]
test_sentiments = sentiments[35000:]

# normalize datasets
norm_train_reviews = pre_process_corpus(train_reviews)
norm_test_reviews = pre_process_corpus(test_reviews)

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [4]:
#Extract features from positive and negative reviews
from sklearn.feature_extraction.text import TfidfVectorizer

# consolidate all normalized reviews
norm_reviews = norm_train_reviews
# get tf-idf features for only positive reviews
positive_reviews = [review for review, sentiment in zip(norm_reviews, sentiments) if sentiment == 'positive']
ptvf = TfidfVectorizer(use_idf=True, min_df=0.02, max_df=0.75, ngram_range=(1, 2), sublinear_tf=True)
tfidf_features = tfidf.fit_transform(documents)
# get tf-idf features for only negative reviews
negative_reviews = [review for review, sentiment in zip(norm_reviews, sentiments) if sentiment == 'negative']
ntvf = TfidfVectorizer(use_idf=True, min_df=0.02, max_df=0.75, ngram_range=(1, 2), sublinear_tf=True)
ntvf_features = ntvf.fit_transform(negative_reviews)
# view feature set dimensions
print(ptvf_features.shape, ntvf_features.shape)

(17490, 893) (17510, 912)


In [5]:
# Topic Modeling
import pyLDAvis
import pyLDAvis.sklearn
from sklearn.decomposition import NMF
import topic_model_utils as tmu

pyLDAvis.enable_notebook()
total_topics = 10

In [6]:
# build topic model on positive sentiment review features
pos_nmf = NMF(n_components=total_topics, solver='cd', max_iter=500,
               random_state=42, alpha=.1, l1_ratio=.85)
pos_nmf.fit(ptvf_features)      
# extract features and component weights
pos_feature_names = np.array(ptvf.get_feature_names())
pos_weights = pos_nmf.components_

# extract and display topics and their components
pos_feature_names = np.array(ptvf.get_feature_names())
feature_idxs = np.argsort(-pos_weights)[:, :15]
topics = [pos_feature_names[idx] for idx in feature_idxs]
for idx, topic in enumerate(topics):
    print('Topic #'+str(idx+1)+':')
    print(', '.join(topic))
    print()

Topic #1:
but, one, no, two, life, even, way, man, time, much, also, us, story, world, like

Topic #2:
movie, movies, really, good, like, but, think, watch, see, movie not, would, great, people, know, say

Topic #3:
show, series, episode, episodes, tv, season, shows, television, watch, watching, like, would, characters, great, think

Topic #4:
love, story, beautiful, wonderful, life, loved, heart, romantic, romance, fall, true, falls, girl, song, woman

Topic #5:
best, ever, seen, one best, one, ever seen, movies, amazing, made, greatest, not seen, never, films, every, brilliant

Topic #6:
great, cast, excellent, acting, well, good, performance, role, story, actors, performances, job, also, character, actor

Topic #7:
film, films, film not, see, watch, horror, film but, cinema, great film, made, but, like, really, recommend, enjoy

Topic #8:
saw, years, dvd, first, ago, time, remember, years ago, saw movie, first time, still, old, since, video, tv

Topic #9:
funny, laugh, hilarious, jo

In [7]:
pyLDAvis.sklearn.prepare(pos_nmf, ptvf_features, ptvf, mds='mmds')


C:\Users\Sm\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
C:\Users\Sm\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
C:\Users\Sm\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.151301  0.121407       1        1  35.253082
1      0.007134  0.091356       2        1  15.844057
5     -0.247790  0.285973       3        1  12.149499
6      0.191281 -0.168057       4        1   8.141104
7      0.416520  0.001167       5        1   7.697361
2      0.241044  0.157048       6        1   5.626878
4      0.150332  0.462094       7        1   5.455793
3      0.134044 -0.435688       8        1   4.571454
8     -0.454753 -0.129814       9        1   2.802220
9     -0.286510 -0.385486      10        1   2.458552, topic_info=             Term         Freq        Total Category  logprob  loglift
268          film  2601.000000  2601.000000  Default  30.0000  30.0000
298         funny  1657.000000  1657.000000  Default  29.0000  29.0000
128        comedy  1536.000000  1536.000000  Default  28.0000  28.0000
444          love  1838.000000  1838.000000  Default  27.0000  27.0000
70           best  1219.000000  1219.000000  Default  26.0000  26.0000
..            ...          ...          ...      ...      ...      ...
758         sweet     5.106077   102.619724  Topic10  -6.3187   0.7050
786         three     4.840995   108.581042  Topic10  -6.3720   0.5952
814           two    10.211987   232.410476  Topic10  -5.6255   0.5807
212  entertaining     7.883419   215.407211  Topic10  -5.8843   0.3978
431        little     5.425321   260.291778  Topic10  -6.2580  -0.1651

[510 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1         1  0.100517  absolutely
1         2  0.164002  absolutely
1         3  0.100517  absolutely
1         5  0.063485  absolutely
1         6  0.037033  absolutely
...     ...       ...         ...
889       4  0.070126         yet
891       1  0.603751       young
891       3  0.193569       young
891       5  0.041479       young
891       8  0.156699       young

[1213 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 6, 7, 8, 3, 5, 4, 9, 10])